# A basic model to try and price the value of an asset

In [19]:
import numpy as np
import pandas as pd
from scipy.optimize import root
import matplotlib.pyplot as plt

Starting Parameters

In [20]:
R = 0.055
b = 1/(1+R)
maturity_g = 0.073
time_frame = 10
start_earnings = 10


## Import the Bloomberg data from Excel

In [21]:
asset_df = pd.read_excel("../../data/Bloomberg_Rankings.xlsx", sheet_name = "Mid Cap and Above")

In [22]:
asset_df.head()

,ID,Name,Last Price,6Y Total Return%,6Y Annualised Return,EPS,FCFPS,FCF Yield,EPS Growth,5 YR EPS Growth,5 YR Forecast EPS Growth,Multiple,Earnings Yield,EPS 5 Years Forecast,5Y Forecast Earnings Yield,1Y Expected TR (No Change in Multiple),5Y Expected Annualised TR (No Change in Multiple)
0,IHC UH Equity,International Holding Co PJSC,410.00,300.470588,1.589450,13.526731,4.789175,0.011681,88.549203,336.721739,336.721739,43.264225,0.023114,21489.034753,52.412280,3.468160,4.891563
1,QNBFB TI Equity,QNB Finansbank AS,328.75,72.222393,1.045380,10.867053,NaN,NaN,69.538759,92.741867,92.741867,30.251914,0.033056,289.062723,0.879278,0.991131,1.212056
2,ADANIGR IN Equity,Adani Green Energy Ltd,2038.00,63.698413,1.003621,6.944302,-74.834324,-0.036719,12.776022,NaN,NaN,231.698702,0.004316,NaN,NaN,NaN,NaN
3,ADANI IN Equity,Adani Power Ltd,874.50,46.398374,0.902365,54.003407,45.250976,0.051745,94.187008,NaN,NaN,16.277971,0.061433,NaN,NaN,NaN,NaN
4,ADE IN Equity,Adani Enterprises Ltd,3645.25,43.643866,0.883477,28.427895,-131.325263,-0.036026,30.473393,52.594889,52.594889,137.441817,0.007276,235.203967,0.064523,0.537051,0.662944


The multiples assume that growth continues at the current rate for a certain set of years and then trails back down to maturity

In [23]:
asset_df = asset_df.dropna(subset=['5 YR Forecast EPS Growth'])


In [24]:
asset_df["5 YR Forecast EPS Growth"]

0       336.721739
1        92.741867
4        52.594889
5        80.841010
6        63.130366
           ...    
1033     12.294384
1039    -10.572361
1042      8.237525
1045     57.864844
1047     25.676972
Name: 5 YR Forecast EPS Growth, Length: 709, dtype: float64

In [25]:

def predict_multiple(growth, t): 
    growth = min([growth/100,0.30])
    maturity_g = 0.065 + 0.1*(growth - 0.065)
    k_vec_m = 1 + maturity_g
    maturity_multiple = (k_vec_m*b*(1-(k_vec_m*b)**18)/(1-b*k_vec_m))
    maturity_multiple   
    b_vec = [b] * t
    k_vec = [1 + growth] * t
    b_vec = np.cumprod(b_vec)[:t]
    k_vec = np.cumprod(k_vec)[:t]
   
    return sum(k_vec*b_vec) + maturity_multiple*b_vec[-1]*k_vec[-1]


predict_multiple(341.92,5)

79.49405534513394

In [26]:
asset_df["5Y Multiple"] = asset_df["5 YR Forecast EPS Growth"].apply(lambda x: predict_multiple(x, 5))


In [27]:
def predict_roi(growth, multiple, fut_multiple): 
    growth = min([growth/100,0.30])
    return (((1+growth)**6)*(1+(1/multiple)+(fut_multiple/multiple)))**1/6


## Predict the ROI Corresponding to this

In [28]:

asset_df["5Y Expected Annualised TR (Change in Multiple)"] = [predict_roi(asset_df["5 YR Forecast EPS Growth"][i], asset_df["Multiple"][i], asset_df["5Y Multiple"][i]) for i in asset_df.index]

In [29]:
asset_df.head()

,ID,Name,Last Price,6Y Total Return%,6Y Annualised Return,EPS,FCFPS,FCF Yield,EPS Growth,5 YR EPS Growth,5 YR Forecast EPS Growth,Multiple,Earnings Yield,EPS 5 Years Forecast,5Y Forecast Earnings Yield,1Y Expected TR (No Change in Multiple),5Y Expected Annualised TR (No Change in Multiple),5Y Multiple,5Y Expected Annualised TR (Change in Multiple)
0,IHC UH Equity,International Holding Co PJSC,410.000000,300.470588,1.589450,13.526731,4.789175,1.168091e-02,88.549203,336.721739,336.721739,43.264225,0.023114,21489.034753,52.412280,3.468160,4.891563,79.494055,2.301199
1,QNBFB TI Equity,QNB Finansbank AS,328.750000,72.222393,1.045380,10.867053,NaN,NaN,69.538759,92.741867,92.741867,30.251914,0.033056,289.062723,0.879278,0.991131,1.212056,79.494055,2.944991
4,ADE IN Equity,Adani Enterprises Ltd,3645.250000,43.643866,0.883477,28.427895,-131.325263,-3.602641e-02,30.473393,52.594889,52.594889,137.441817,0.007276,235.203967,0.064523,0.537051,0.662944,79.494055,1.275612
5,SMCI US Equity,Super Micro Computer Inc,762.490723,30.004074,0.772433,19.230000,-33.824639,-4.436072e-02,72.776280,80.841010,80.841010,42.665360,0.023438,371.932307,0.487786,0.850796,1.045355,79.494055,2.322208
6,BYAN IJ Equity,Bayan Resources Tbk PT,18000.000000,22.218526,0.689037,0.034590,0.017968,9.982250e-07,-46.202778,63.130366,63.130366,32.120832,0.031132,0.399599,0.000022,0.682090,0.810109,79.494055,2.820447


In [30]:
asset_df.to_excel("../../data/Top_Stocks.xlsx")